# Romance

In [1]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "romance"
method = "self_translation"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

def get_gpt_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_gpt_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


/Users/aryan/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Original Claim: Le Nigeria dépense 7 fois moins en soins de santé par habitant que l'Afrique du Sud.
Model Translated Claim: Nigeria spends 7 times less on healthcare per capita than South Africa.
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: False
Wrong response


Original Claim: Le quart bloquant a été supprimé de la Constitution sénégalaise. Il n'existe plus depuis 2006.
Model Translated Claim: 'The blocking quarter has been removed from the Senegalese Constitution. It has not existed since 2006.'
Model Output: Final Answer: True
Final Answer: True, Actual Answer: True
Correct response


Original Claim: «Fin quando siamo stati al governo noi, avevamo una disoccupazione inferiore di due punti alla media europea, oggi abbiano una disoccupazione superiore di due punti alla media europea. La disoccupazione giovanile è addirittura drammatica, al 32%; significa che 3 giovani su 10 non hanno un lavoro»
Model Translated Claim: "While we were in government

# Indo-Aryan

In [2]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "indo_aryan"
method = "self_translation"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

def get_gpt_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_gpt_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Original Claim: ছবিটি সম্প্রতি আমেরিকার মিনিয়াপোলিসে পুলিশের অত্যাচারে মৃত্যু হওয়া জর্জ ফ্লয়েডের
Model Translated Claim: 'The image is of George Floyd, who recently died due to police brutality in Minneapolis, America.'
Model Output: Final Answer: True
Final Answer: True, Actual Answer: Mostly false
Wrong response


Original Claim: इस वीडियो में निर्माता-निर्देशक महेश भट्ट फिल्म ‘सड़क 2’ के ट्रेलर को बड़े पैमाने पर डिस्लाइक किए जाने की वजह से नाराजगी जता रहे हैं.
Model Translated Claim: In this video, producer-director Mahesh Bhatt is expressing his displeasure over the large-scale disliking of the trailer for the film 'Sadak 2'.
Model Output: Final Answer: True
Final Answer: True, Actual Answer: False
Wrong response


Original Claim: सर्वोच्च न्यायालयाने 15 जून पासून देशाचे नाव प्रत्येक भाषेत फक्त भारत असेल असा निर्णय दिला आहे
Model Translated Claim: 'The Supreme Court has ruled that from June 15, the name of the country will be only India in every language.'
Model Output: Final Answe

# Kartvelian

In [3]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "kartvelian"
method = "self_translation"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

def get_gpt_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_gpt_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Original Claim: სახელმწიფო ბიუჯეტის ხარჯვით ნაწილს ლარის გაუფასურებაზე გავლენა არ ჰქონია
Model Translated Claim: 'The expenditure part of the state budget has not been affected by the depreciation of the lari.'
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly false
Wrong response


Original Claim: აჭარაში 25 ათასი ბათილი ბიულეტენი არ დაფიქსირებულა
Model Translated Claim: 'In Adjara, 25 thousand invalid ballots were not recorded.'
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response


Original Claim: რუსთავი 2-ის წინააღმდეგ ვიხილეთ უკანონო, კოორდინირებული მოქმედება ორი შტოსი - სასამართლოს და საჯარ...
Model Translated Claim: "We witnessed illegal, coordinated action against Rustavi 2 by two branches - the judiciary and the public..."
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: True
Wrong response


Original Claim: „სასამართლო დავების დროს, როდესაც ბიზნესი და სახელმწიფო დავობდ

# Slavic

In [4]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "slavic"
method = "self_translation"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

def get_gpt_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_gpt_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Original Claim: „I u drugim evropskim zemljama postoje različite akcize na različit kvalitet naftnih derivata.“
Model Translated Claim: "In other European countries, there are also different excise taxes on various qualities of petroleum products."
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly false
Wrong response


Original Claim: “Što se tiče tramvaja, oni nisu mogli da prođu zbog okupljenih građana, a saobraćaj je normalizovan posle završetka protesta.”
Model Translated Claim: "As for the trams, they could not pass due to the gathered citizens, and traffic was normalized after the protest ended."
Model Output: Final Answer: True
Final Answer: True, Actual Answer: False
Wrong response


Original Claim: Медианный размер средней зарплаты в январе составлял 112 тыс тенге. А модальный, то есть самый распространенный размер зарплаты, составляет 58 тыс тенге.
Model Translated Claim: "The median size of the average salary in January was 112,000 ten

# Turkic

In [5]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "turkic"
method = "self_translation"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

def get_gpt_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_gpt_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Original Claim: biz Avrupalılardan 42 dolar daha fazla para veriyoruz doğalgaza
Model Translated Claim: 'We pay 42 dollars more for natural gas than Europeans.'
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: Mostly false
Wrong response


Original Claim: There are no problems in the religious sphere in Azerbaijan, religious freedom is fully ensured
Model Translated Claim: There are no problems in the religious sphere in Azerbaijan; religious freedom is fully ensured.
Model Output: Final Answer: Mostly False
Final Answer: Mostly false, Actual Answer: Mostly false
Correct response


Original Claim: 2023 senesine yaklaşırken Türkiye, dünyanın en büyük 10 ekonomisi olması gibi bir hedef koymuştuk. Bugünlerde ilk 20’den dışarı çıkma tehlikesiyle karşı karşıyayız.
Model Translated Claim: "As we approach the year 2023, we had set a goal for Turkey to be among the world's top 10 economies. Nowadays, we are facing the risk of falling out of the top 20."
Model Outpu